In [11]:
import numpy as np
import pandas as pd


In [12]:
def validate_8bit_adder_no_cin(adder_data, start_time=2.5e-9, interval=5e-9):
    print(f"{'Time':<12} {'a':<26} {'b':<26} {'Sum (Actual)':<26} {'Sum (Expected)':<26}{'Carry_out (Actual)':<5}  {'Carry_out (Expected)':<5}")
    print("-" * 177)
    issues = []
    eval_times = np.arange(start_time, adder_data['/a0 X'].max(), interval)

    for t in eval_times:
        # Identify rows surrounding the evaluation time
        prior_row = adder_data[adder_data['/a0 X'] <= t].iloc[-1]
        next_row = adder_data[adder_data['/a0 X'] > t].iloc[0]

        # Linearly interpolate signal values
        interpolated_vals = {}
        for col in adder_data.columns:
            if ' X' not in col:  # Skip the time-related columns
                t1, t2 = prior_row['/a0 X'], next_row['/a0 X']
                v1, v2 = prior_row[col], next_row[col]
                interpolated_vals[col] = v1 + (v2 - v1) * (t - t1) / (t2 - t1)

        # Convert signal values to binary (threshold: >0.5 is 1, otherwise 0)
        a_signals = [int(interpolated_vals[f'/a{i} Y'] > 0.5) for i in range(8)]
        b_signals = [int(interpolated_vals[f'/b{i} Y'] > 0.5) for i in range(8)]
        sum_signals = [int(interpolated_vals[f'/s{i} Y'] > 0.5) for i in range(8)]
        carry_out = int(interpolated_vals['/cout Y'] > 0.5)

        # Logic for carry generation and propagation
        generate = [a_signals[i] & b_signals[i] for i in range(8)]
        propagate = [a_signals[i] ^ b_signals[i] for i in range(8)]
        carries = [0]  # No carry-in, so the initial carry is 0

        # Calculate carries and sums
        for i in range(8):
            carries.append(generate[i] | (propagate[i] & carries[i]))

        expected_sums = [propagate[i] ^ carries[i] for i in range(8)]
        expected_carry_out = carries[8]

        # Print the table row
        print(f"{t:<12.2e} {str(a_signals):<26} {str(b_signals):<26} {str(sum_signals):<26}{str(expected_sums):<26} {carry_out:<20}  {expected_carry_out:<20}")

        # Calculate carries and sums
        for i in range(8):
            carries.append(generate[i] | (propagate[i] & carries[i]))

        # Check sum outputs and final carry-out
        for i in range(8):
            expected_sum = propagate[i] ^ carries[i]
            if sum_signals[i] != expected_sum:
                issues.append((t, f'/s{i}', sum_signals[i], expected_sum))
        if carries[8] != carry_out:
            issues.append((t, '/cout', carry_out, carries[8]))

    return issues
# Main script for loading and validating 8-bit adder logic
print("Validating 8-Bit Adder Logic and Printing States...")
try:
    # Load the CSV file containing simulation data
    adder_data = pd.read_csv("Simulations Data/8bit_addr.csv")
    errors = validate_8bit_adder_no_cin(adder_data)

    # Report findings
    if errors:
        print("Issues detected in 8-bit adder logic:")
        for error in errors:
            print(f"Time: {error[0]:.2e}, Signal: {error[1]}, Actual: {error[2]}, Expected: {error[3]}")
    else:
        print("8-bit adder logic verified successfully. No issues detected.")
except FileNotFoundError:
    print("Error: File '8bit_addr.csv' not found. Please check the file's location.")


Validating 8-Bit Adder Logic and Printing States...
Time         a                          b                          Sum (Actual)               Sum (Expected)            Carry_out (Actual)  Carry_out (Expected)
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
2.50e-09     [0, 0, 0, 0, 0, 0, 0, 0]   [0, 0, 0, 0, 0, 0, 0, 0]   [0, 0, 0, 0, 0, 0, 0, 0]  [0, 0, 0, 0, 0, 0, 0, 0]   0                     0                   
7.50e-09     [0, 0, 0, 0, 0, 0, 0, 0]   [0, 0, 0, 0, 0, 0, 0, 0]   [0, 0, 0, 0, 0, 0, 0, 0]  [0, 0, 0, 0, 0, 0, 0, 0]   0                     0                   
1.25e-08     [0, 0, 0, 0, 0, 0, 0, 0]   [0, 0, 0, 0, 0, 0, 0, 0]   [0, 0, 0, 0, 0, 0, 0, 0]  [0, 0, 0, 0, 0, 0, 0, 0]   0                     0                   
1.75e-08     [0, 0, 1, 0, 0, 1, 0, 0]   [0, 1, 0, 1, 0, 1, 0, 1]   [0, 1, 1, 1, 0, 0, 1, 1]  [0, 1, 1, 1, 0, 0, 1, 1]   